### Fetch daily reports data from Github and update Bigquery table
- Github repository https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports
- John Hopkins dashboard https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6
- Always the fetch the lates csv (yesterday)

In [ ]:
from datetime import timedelta, date, datetime
from collections import Counter
import pandas as pd, numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage

### Bigquery credentials
https://googleapis.dev/python/bigquery/latest/index.html
- Create a google project
- Create a bigquery dataset in this project
- Create table called "daily_reports"

In [ ]:
cred_json = 'Directory of google api credential json'
project_id = 'Google project id'
project_name = 'Google project name'
dataset = 'name of the corresponding Bigquery dataset'
credentials = service_account.Credentials.from_service_account_file(cred_json)
client = bigquery.Client(project = project_id,credentials = credentials)

In [ ]:
QUERY = "SELECT * from "+project_name+'.'+"daily_reports ORDER BY Last_Update DESC LIMIT 1"
query_job = client.query(QUERY)
df_last = query_job.to_dataframe()
df_last

In [ ]:
latest_day = df_last['Last_Update'].iloc[-1].date()
yesterday = (date.today() - timedelta(days=1))
isupdate = False if (yesterday-latest_day).days>0 else True

In [ ]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)
        
def parse_date(date_):
    if 'T' in date_:
        res = datetime.strptime(date_, '%Y-%m-%dT%H:%M:%S')
    elif len(date_.split(' ')[0])<8:
        res = datetime.strptime(date_,'%m/%d/%y %H:%M')
    else:
        res =  datetime.strptime(date_, '%m/%d/%Y %H:%M')
    return res       

### Fetching data from Github repository and process it

In [ ]:
base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
if  not isupdate:
    # Delete everything in the table first
    dml_statement = (
        "DELETE coronavirus_dashboard_data.daily_reports WHERE TRUE "
        )
    query_job = client.query(dml_statement)  # API request
    query_job.result()
    yesteday = (date.today() - timedelta(days=1)).strftime("%m-%d-%Y")
    url = base_url+yesteday+'.csv'
    df = pd.read_csv(url)
else:
    print('daily_reports table already up to date')

In [ ]:
# Formating the columns
df = df.drop(columns=['FIPS','Admin2','Combined_Key'])
df = df.rename(columns={'Province_State':'Province/State','Country_Region':'Country/Region','Last_Update':'Last Update','Lat':'Latitude','Long_':'Longitude'})
cols = ['Province/State','Country/Region','Last Update','Confirmed','Deaths','Recovered','Latitude','Longitude']
df = df[cols]
df['Country/Region']=df['Country/Region'].apply(lambda x: 'China' if 'Mainland' in x else x)
df['Province/State']=df.apply(lambda x: x['Province/State'] if isinstance(x['Province/State'],str) else x['Country/Region'], axis=1)

In [ ]:
# Drop duplictes and empty records
df_new = df.drop_duplicates()
df_new = df_new.dropna(subset=['Confirmed','Deaths','Recovered'],how='all')
print(len(df),len(df_new))

In [ ]:
df_new.to_csv('data_dailyreport.csv', index = None, header=True)

### Write to Bigquery table

In [ ]:
table_id = project_id+'.'+dataset+'.daily_reports'
table = client.get_table(table_id) 

In [ ]:
if not isupdate:
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
    )

    with open('data_dailyreport.csv', "rb") as source_file:
        job = client.load_table_from_file(source_file, table_id, job_config=job_config)

    job.result()  # Waits for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )